# <b>Problem 1</b>

<b>To create the above dataframe:</b>

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

installing required packages

In [31]:
!pip install beautifulsoup4
!pip install lxml 
print("in my noteboook they are already installed")

in my noteboook they are already installed


In [24]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings

<b>Scrape the List of postal codes of Canada</b>

In [3]:
canada_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(canada_url).text #requesting to get url

In [4]:
soup = BeautifulSoup(source, 'xml')

In [5]:
table=soup.find('table')

In [9]:
from IPython.display import display_html #used to display html table 
tab = str(soup.table)
#display_html(tab,raw=True)

The html table is converted to Pandas DataFrame for cleaning and preprocessing.

In [12]:
dfs=pd.read_html(tab)
df=dfs[0]

In [13]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<b>Data preprocessing and cleaning</b>

In [16]:
df_new = df[df.Borough != 'Not assigned']

In [17]:
df_new.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"



Find whether there is a "Not assigned" in Neighbourhood

In [18]:
df_new.loc[df_new.Neighbourhood == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [25]:

df_new.Neighbourhood.replace('Not assigned',df_new.Borough,inplace=True)
df_new.head(8) #ignore the warnings if there are any or use- warnings.filterwarnings("ignore")


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills


Group Neighbourhoods with the same Postcode

In [28]:
df_toronto = df_new.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x))
df_toronto = df_toronto.reset_index()
df_toronto.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
df_toronto.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [29]:
df_toronto.shape

(103, 3)